In [2]:
pip install QuantLib-Python

  Obtaining dependency information for QuantLib-Python from https://files.pythonhosted.org/packages/7e/a9/4c6fac9c3c9b1625ab573531bbe9666d2f7cd6bc0a5eeb1f5792b947d9b2/QuantLib_Python-1.18-py2.py3-none-any.whl.metadata
  Obtaining dependency information for QuantLib from https://files.pythonhosted.org/packages/eb/96/58ad2e21aab7409fca35cbf9d100bd1a9a02a329589e173462884be5ae58/QuantLib-1.35-cp38-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/12.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.5 MB 217.9 kB/s eta 0:00:58
   ---------------------------------------- 0.0/12.5 MB 326.8 kB/s eta 0:00:39
   ---------------------------------------- 0.1/12.5 MB 653.6 kB/s eta 0:00:20
   - -------------------------------------- 0.3/12.5 MB 1.6 MB/s eta 0:00:08
   - -------------------------------------- 0.5/12.5 MB 2.4 MB/s eta 0:00:05
   - -------------------------------------- 

'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import QuantLib as ql


In [14]:

# Market data
spot_price = 100.0  # S0
strike_price = 100.0  # strike
volatility = 0.2  # sigma
risk_free_rate = 0.05  # r
dividend_rate = 0.0  # Assuming no dividends

# Option data
maturity_in_days = 63  # N
calendar = ql.NullCalendar()
day_count = ql.Actual365Fixed()
settlement_date = ql.Date.todaysDate()
ql.Settings.instance().evaluationDate = settlement_date
maturity_date = settlement_date + int(365 * (64/365))  # T in days

option_type = ql.Option.Call

# Fixing dates
fixing_dates = [settlement_date + i for i in range(1, maturity_in_days + 1)]

# Payoff and exercise
payoff = ql.PlainVanillaPayoff(option_type, strike_price)
exercise = ql.EuropeanExercise(maturity_date)

# Average type
average_type = ql.Average.Geometric

# Asian option
option = ql.DiscreteAveragingAsianOption(
    average_type,
    1.0,  # running accumulator for geometric average
    0,    # past fixings
    fixing_dates,
    payoff,
    exercise
)

# Yield and volatility curves
spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
flat_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(settlement_date, risk_free_rate, day_count))
dividend_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(settlement_date, dividend_rate, day_count))
vol_ts = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(settlement_date, calendar, volatility, day_count))

# Black-Scholes-Merton process
bsm_process = ql.BlackScholesMertonProcess(
    spot_handle, dividend_ts, flat_ts, vol_ts)

# Pricing engine
engine = ql.AnalyticDiscreteGeometricAveragePriceAsianEngine(bsm_process)
option.setPricingEngine(engine)

# Calculate and print the option price
option_price = option.NPV()
print(f"The price of the Asian geometric call option is: {option_price:.4f}")


The price of the Asian geometric call option is: 2.1182


In [18]:
option.delta()

0.5382336079671503

In [15]:
from DeepHedging.Agents import AsianDeltaHedgingAgent
from DeepHedging.HedgingInstruments import GBMStock
from DeepHedging.ContingentClaims import AsianGeometricCall
from DeepHedging.CostFunctions import ProportionalCost
from DeepHedging.RiskMeasures import CVaR

T = 64/365
N = 64
r = 0.05

instrument1 = GBMStock(S0=100, T=T, N=N, r=r, sigma=0.2)

instruments = [instrument1] #instrument1
contingent_claim = AsianGeometricCall(strike=100)

path_transformation_configs = [
    {'transformation_type': 'log_moneyness', 'K': contingent_claim.strike}#,
    #{'transformation_type': None}
]

cost_function = ProportionalCost(proportion=0.0)
risk_measure = CVaR(alpha=0.5)

delta_agent = AsianDeltaHedgingAgent(instrument1, contingent_claim)
bs_price = delta_agent.get_model_price()
print('p:', bs_price)


p: tf.Tensor(2.1107864, shape=(), dtype=float32)
